## Arquitectura IoT

![01.png](img/01.png)

![02.png](img/02.png)


### Libreria network, forma de uso

```
// Libreria
import network

// Objeto
wf = network.WLAN(tipo)
# tipo puede ser network.STA_IF, network.AP_IF

// Funciones
wf.active(True) # False desactivar
wf.scan() # devuelve una lista
wf.connect(ssid, password)
wf.isconnected() # Devuelve True o False
wf.config("mac") # devuelve la mac
wf.ifconfig() # devuelve ip, netmask, gw, DNS

```

## Ejemplo 01 Escaneo de todas las redes wifi disponibles


#### ahora si, dibujando la espiral
**06_ejemplo_01_scan_wifi.py**


***
```Python

import network

wf = network.WLAN(network.STA_IF)
wf.active(True)
lista=wf.scan()

for k in lista:
    print(k) # mac, ,nivel de señal, ,

```
***

<video width="320" height="240" controls>
  <source src="video/01.mp4" type="video/mp4">
Your browser does not support the video tag.
</video>

## Ejemplo 02 Conectandose a la red WiFi


#### ahora si, dibujando la espiral
**06_ejemplo_02_conexion_wifi.py**


***
```Python

import network
import utime


wf = network.WLAN(network.STA_IF)
wf.active(True)

wf.connect('nombre SSID', 'clave de la red')

# la conexión puede demorar unos segundos,
# Creamos un ciclo que termina cuando se ha realizado la conexión
while not wf.isconnected():
    print(".")
    utime.sleep(1) # cada segundo se imprime un punto

# al salir, imprimimos
print("Conectado al WiFi")
print(wf.ifconfig())


```
***

![03.png](img/03.png)

A partir de este momento se puede hacer ping al dispositivo raspberry pi pico w


## Sockets Cliente-Servidor


 La raspberry puede ser cliente o puede ser servidor.
 
![04.png](img/04.png) 

###  Funciones Sockets cliente servidor

***
```
//Librería
import usocket

//Objeto
s=usocket.socket(usocket.AF_INET,usocket.SOCK_STREAM) # AF_INET socket IPv4, SOCK_STREAM puede ser TCP o UDP
s=usocket.socket()

// Funciones servidor (crear un servidor)
s.bind(("ip", puerto))
s.listen(c) # indicar el numero de conexiones simultáneas permitidas
(sc,addr)=s.accept() # crea un socket cuando el cliente se conecta al servidor, se queda bloqueado con s.accept()
                     # sc es el socket del cliente y addr es la dirección de ese cliente.
                     
// Funciones cliente
s.connect((ip, puerto)) # conectarse a un servidor.

// Funciones cliente-servidor, la variable "s" y la variable "sc" definen sockets
s.send(data)
dato=s.recv(buffer)
s.close()

```
***

Micropython es similar a Python, asi que lo anterior es válido para python, pero no es usocket, sino socket. Se hace la precisión debido a que ahora se necesitan dos elementos en la red de computadoras, por ejemplo, la compuradora y la raspberry pi pico.

Se sugiere instalar otro IDE para trabajar la parte que se programará en la computadora, por ejemplo PyCharm. **En el caso de este ejemplo se usará [PyCharm](https://www.jetbrains.com/pycharm/buy/?fromIDE=&lang=en-us&section=commercial&billing=yearly). **





## Ejemplo 03 Crear un programa en Python que se conecte con un servidor ejecutandose en las Raspberry Pi pico para enviar mensajes


### Servidor en la raspberry pi pico

**06_ejemplo_03_servidor_pico.py**

***
```Python
ssid='xxxxx'
claveSSID = 'xxxxxx'

import network
import utime
import usocket
from machine import Pin # ejemplo led

## CONEXION A LA RED

wf = network.WLAN(network.STA_IF)
wf.active(True)

wf.connect(ssid, claveSSID)

# la conexión puede demorar unos segundos,
# Creamos un ciclo que termina cuando se ha realizado la conexión
while not wf.isconnected():
    print(".")
    utime.sleep(1) # cada segundo se imprime un punto

# al salir, imprimimos
print("Conectado al WiFi")
print(wf.ifconfig())

## CREACION DEL SERVIDOR
s=usocket.socket()
led=Pin(18, Pin.OUT)  # ejemplo led
s.bind(("192.168.2.164", 2025))
s.listen(10)
print("Socket creado, esperando conexiones...")
(sc,addr)=s.accept()
print("Cliente conectado, IP:", addr)
continuar=True

while continuar:
    dato = sc.recv(32).decode() # se decodifica para pasarlo a string
    print(dato)
    if dato == "z":
        continuar=False
    elif dato == "A": # ejemplo led
        led.on() # ejemplo led
    elif dato == "B": # ejemplo led
        led.off() # ejemplo led       
print("Fin del programa")
sc.close()
s.close()








```
***

### Cliente en la computadora (Pycharm)

**ClientePython.py**

***
```Python
import socket

s=socket.socket()
print("Por conectarse al servidor")
s.connect(("192.168.2.164", 2025))
print("Conectado al servidor")
continuar=True

while continuar:
    a=input("Digite el mensaje a enviar: ")
    s.send(a.encode())
    if a == "z":
        continuar=False
s.close()

```
***

<video width="320" height="240" controls>
  <source src="video/02.mp4" type="video/mp4">
Your browser does not support the video tag.
</video>

## Ejemplo 04 Crear un programa en Python con 2 botones que encienda y apague un led vía dirección IP, conectado al GP 18

### Graficos de Python

***
```
// Librería
from tkinter import Tk
from tkinter import Button

// Objeto
tk = Tk()
b1 = Button(tk, text="Boton", command=funcion)

// Funciones
Tk.mainloop() # bucle, siempre poner al final
Tk.geometry("largoXancho") # Tamaño de la ventana
b1.place(x=250,y=100) # posición de la pantalla en donde aparecerá.
```
***

### Primero creamos el servidor en el raspberry pi pico

**06_ejemplo_04_servidor_pico_led.py**

***
```Python
ssid='xxxxx'
claveSSID = 'xxxxx'

import network
import utime
import usocket
from machine import Pin # ejemplo led

## CONEXION A LA RED

wf = network.WLAN(network.STA_IF)
wf.active(True)

wf.connect(ssid, claveSSID)

# la conexión puede demorar unos segundos,
# Creamos un ciclo que termina cuando se ha realizado la conexión
while not wf.isconnected():
    print(".")
    utime.sleep(1) # cada segundo se imprime un punto

# al salir, imprimimos
print("Conectado al WiFi")
print(wf.ifconfig())

## CREACION DEL SERVIDOR
s=usocket.socket()
#led=Pin(18,Pin.OUT)  # ejemplo led
led=Pin("LED",Pin.OUT)  # ejemplo led
s.bind(("192.168.2.164", 2025))
s.listen(10)
print("Socket creado, esperando conexiones...")
(sc,addr)=s.accept()
print("Cliente conectado, IP:", addr)
continuar=True

while continuar:
    dato = sc.recv(32).decode() # se decodifica para pasarlo a string
    print(dato)
    if dato == "z":
        continuar=False
    elif dato == "A": # ejemplo led
        led.on() # ejemplo led
    elif dato == "B": # ejemplo led
        led.off() # ejemplo led       
print("Fin del programa")
sc.close()
s.close()






```
***


### Segundo creamos la ventana con los botones (aún sin funcionalidad en los botones)

**ClienteBotones.py**

***
```Python
from tkinter import Tk
from tkinter import Button
tk=Tk()

b1=Button(tk, text="Encender")
b2=Button(tk, text="Apagar")

tk.geometry("300x200")
b1.place(x=100,y=50)
b2.place(x=200,y=50)


tk.mainloop()

```
***


### Tercero agregamos los sockets y la funcionalidad a los botones 

**ClienteBotones.py**

***
```Python

from tkinter import Tk
from tkinter import Button
import socket

def encender():
    # print("Presionó encender")
    s.send("A".encode())

def apagar():
    # pass
    s.send("B".encode())


### FUNCIONALIDAD INTERFAZ GRAFICA
tk=Tk()
b1=Button(tk, text="Encender", command=encender)
b2=Button(tk, text="Apagar", command=apagar)
tk.geometry("300x200")
b1.place(x=100,y=50)
b2.place(x=200,y=50)

### AGREGANDO SOCKETS
s=socket.socket()
print("Por conectarse al servidor")
s.connect(("192.168.2.164", 2025))
print("Conectado al servidor")

tk.mainloop()

```
***

<video width="320" height="240" controls>
  <source src="video/03.mp4" type="video/mp4">
Your browser does not support the video tag.
</video>



## Probar desde el teléfono 

Buscar la App **TCP Client** en el **PlayStore**  de Google, de ELX Device.

Indicar la dirección IP y el puerto, mientras se ejecuta la aplicacion en la raspberry pi pico, luego dar clic en el botón de conect y enviar la letra **A** y luego la letra **B**

![05.jpeg](img/05.jpeg)